<a href="https://colab.research.google.com/github/ikabrain/UML501-IKANSH-MAHAJAN-102303754-LabAssignments/blob/main/ML_assign4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Assignment 4
---

In [65]:
import numpy as np
import pandas as pd

## Q1. Write a Python program to scrape all available books from the website [Books to Scrape]((https://books.toscrape.com/)) - a live site built for practicing scraping (safe, legal, no anti-bot).
---

For each book, extract the following details:
1.   Title
2.   Price
3.   Availability (In stock / Out of stock)
4.   Star Rating (One, Two, Three, Four, Five)

Store the scraped results into a Pandas DataFrame and export them to a CSV file named books.csv.

(Note: Use the requests library to fetch the HTML page. Use BeautifulSoup to parse and extract book details and handle pagination so that books from all pages are scraped)

In [66]:
from bs4 import BeautifulSoup
import requests

In [67]:
BASE_URL = "https://books.toscrape.com/catalogue/category/books_1/page-{}.html"

In [68]:
books = []
page = 1

while True:
    try:
        response = requests.get(BASE_URL.format(page))
        html = response.text # response.content can be used to get HTML as a binary instead of an Unicode string
        soup = BeautifulSoup(html, 'lxml')
    except Exception as e:
        print(f"ERROR: {e}")
        break

    book_tags = soup.find_all('article', class_='product_pod')
    if not book_tags or not response.ok:
        break

    for book_tag in book_tags:
        book = {
            'title': book_tag.h3.a['title'],
            'price': float(book_tag.find('p', class_='price_color').text[2:]),
            'availability': book_tag.find('p', class_='instock availability').text.strip(),
            'star_rating': book_tag.find('p', class_='star-rating')['class'][1]
        }
        books.append(book)

    page = page + 1

df = pd.DataFrame(books)
df.head()

,title,price,availability,star_rating
0,A Light in the Attic,51.77,In stock,Three
1,Tipping the Velvet,53.74,In stock,One
2,Soumission,50.10,In stock,One
3,Sharp Objects,47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,54.23,In stock,Five


In [69]:
df.to_csv('books.csv', index=False)

## Q2. Write a Python program to scrape the [IMDB Top 250 Movies list](https://www.imdb.com/chart/top/).
---

For each movie, extract the following details:

1.   Rank (1-250)
2.   Movie Title
3.   Year of Release
4.   IMDB Rating

Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv

(Note: Use Selenium/Playwright to scrape the required details from this website)

In [70]:
!pip install selenium > /dev/null
!apt install -y chromium-chromedriver > /dev/null

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [72]:
# Setting up & installing Chrome
options = Options()
options.add_argument("--headless=new")          # Run Chrome in headless mode (without a GUI)
options.add_argument("--no-sandbox")            # Required in Colab
options.add_argument("--disable-dev-shm-usage") # Avoid limited /dev/shm space

In [75]:
URL = "https://www.imdb.com/chart/top/"

In [76]:
driver = webdriver.Chrome(options=options)  # Opens a Chromium browser
driver.get(URL) # Loads the page

# Wait until the list is visible
wait = WebDriverWait(driver, 10)
try:
    movies = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.chart"))
    )
    print(movies)
except Exception as e:
    print(f"Page load timeout: {e}")
    print(driver.page_source[:1000])
finally:
    driver.quit()

[<selenium.webdriver.remote.webelement.WebElement (session="aa88e2d9cd962eb23c038f214e6f0693", element="f.4DCAFBB4133B251062C77885C1793E92.d.FE9845222B6235E4A6D22E629FDA763F.e.57")>]


## Q3. Write a Python program to scrape the weather information for top world cities from the given website [timeanddate.com](https://www.timeanddate.com/weather/).
---


For each city, extract the following details:

1.   City Name
2.   Temperature
3.   Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)

Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

##